# Подготовка датафреймов к загрузке в Базу данных (первичная)

## Импорт библиотек и датафреймов

In [1]:
import pandas as pd
import numpy as np

In [2]:
# импорт csv файлов, которые были подготовлены 
df_heroes = pd.read_csv('df_heroes.csv')
df_matches = pd.read_csv('df_matches.csv')
df_players = pd.read_csv('df_players.csv')
df_items = pd.read_csv('items.csv')

## Предобработка данных

### Предобработка df_matches, df_players

In [3]:
# создадим функцию для нормализации строковых типов данных object
def normalize_object(df):
    """ Принимает df, находит колонки 'object', нормализует их
    убирает заглавные буквы и пробелы в начале и в конце строк
    """
    for col in df.select_dtypes(include=["object"]):
        df[col] = df[col].astype(str).str.strip().str.lower()
    return df


In [4]:
# Нормализуем строковые типы данных с использованием функции normalize_object(df)
df_heroes = normalize_object(df_heroes)
df_items = normalize_object(df_items)
df_players = normalize_object(df_players)

In [5]:
# Преобразуем необходимые колонки в Int54, bool
df_matches['region'] = df_matches['region'].astype('Int64')
df_players['steam_id'] = df_players['steam_id'].astype('Int64')
df_players['win'] = df_players['win'].astype('bool')

### Предобработка df_items

In [6]:
# Преобразуем типы данных в df_items
df_items['cost']  = df_items['cost'] .astype('Int64')
df_items['bkbpierce'] = df_items['bkbpierce'].astype('bool')

In [7]:
# Заменим '[]' на NA в df_items['hint']
df_items.loc[df_items['hint'].astype(str).str.strip() == '[]', 'hint'] = pd.NA

In [8]:
# приводим mc, hc, cd, charges к int, предварительно заменив false на nA
for col in ['mc', 'hc', 'cd', 'charges']:
    df_items[col] = pd.to_numeric(
        df_items[col].replace(['False', False], pd.NA),
        errors='coerce'
    ).astype('Int64')

### Проверка и удаление дубликатов

In [9]:
# удалим дубликаты, если они есть
df_matches = df_matches.drop_duplicates(subset=None, keep='first')
df_players = df_players.drop_duplicates(subset=["match_id", "player_slot"], keep='first').reset_index(drop=True)

In [10]:
# Переименуем колонки, которые будут ключами:
df_heroes = df_heroes.rename(columns={"id": "hero_id"})
df_items = df_items.rename(columns={"id": "item_id"})

## Подготовка данных из df_players к загрузке в базу данных

Разделим df_players на несколько таблиц:  
* player_matches:
    * match_id pk(составной), fk
    * player_slot pk(составной)
    * steam_id
    * personaname
    * hero_id, fk
    * is_radiant
    * win
* player_stat
    * match_id - pk, fk
    * player_slot - pk, fk
    * kills
    * deaths
    * assists
    * denies
    * last_hits
    * gold_per_min
    * xp_per_min
    * hero_damage
    * hero_healing
    * tower_damage
* player_items - для player_items item_0 - n заменим на item_slot, а содержимое заменим на item_id
    * match_id - pk, fk
    * player_slot - pk, fk
    * item_slot - pk 
    * item_id 

In [11]:
# создадим df_player_matches
df_player_matches = df_players[['match_id', 'player_slot', 
                                'steam_id', 'personaname', 'hero_id', 'is_radiant', 'win']]

In [12]:
# создадим df_player_stat
df_player_stat = df_players[['match_id', 'player_slot', 'kills', 'deaths', 
                             'assists', 'denies', 'last_hits', 'gold_per_min', 'xp_per_min', 
                             'hero_damage', 'hero_healing', 'tower_damage']]


In [13]:
# создадим df_player_items 
df_player_items = df_players[['match_id', 'player_slot','item_0', 'item_1', 'item_2', 'item_3', 'item_4', 'item_5']]

# из широкой таблицы сделаем плоскую: slot_col - название столбца, item_id - id предмета
df_player_items = df_player_items.melt(
                            id_vars = ['match_id', 'player_slot'], 
                            value_vars = ['item_0', 'item_1','item_2', 'item_3', 'item_4', 'item_5'], 
                            var_name = 'slot_col', 
                            value_name='item_id')

# создадим столбец, взяв последний элемент строки столбца slot_col 
df_player_items['item_slot'] = df_player_items['slot_col'].str.slice(-1)

# удалим колонку slot_col 
df_player_items = df_player_items.drop(columns='slot_col')

In [14]:
# встречается ноль - это остутствующее значение. Необходимо заменить его на nan, чтобы не было ошибки при стыковке таблиц
df_player_items['item_id'] = df_player_items['item_id'].replace(0, pd.NA)

## Проверка совпадения внешних ключей

### df_matches - df_player_matches

In [15]:
# проверим совпадение внешних ключей между таблицами df_matches, df_player_matches
s1 = set(df_matches['match_id'])
s2 = set(df_player_matches["match_id"])
matches_and_players = s1 - s2
display(len(matches_and_players), len(s1), len(s2))

100

19353

19253

In [16]:
# По какой-то причине часть матчей не прогрузилась
missing_matches = df_matches.loc[df_matches["match_id"].isin(matches_and_players)]
display(len(missing_matches))

100

In [17]:
# Удалим лишние матчи
df_matches = df_matches[df_matches["match_id"].isin(df_player_matches["match_id"])]


In [18]:
len(set(df_player_matches["match_id"]) - set(df_matches["match_id"])) == 0

True

### df_player_matches vs df_player_stat

In [19]:
# проверим совпадение внешних ключей между таблицами df_player_matches vs df_player_stat
s1 = set(df_player_matches[['match_id', 'player_slot']].apply(tuple, axis=1))
s2 = set(df_player_stat[['match_id', 'player_slot']].apply(tuple, axis=1))
players_and_players_stat= s1 - s2
display(len(players_and_players_stat), len(s1), len(s2))

0

192530

192530

### df_player_matches vs df_player_items

In [20]:
# проверим совпадение внешних ключей между таблицами df_player_matches vs df_player_items
s1 = set(df_player_matches[['match_id', 'player_slot']].apply(tuple, axis=1))
s2 = set(df_player_items[['match_id', 'player_slot']].apply(tuple, axis=1))
players_and_player_items= s1 - s2
display(len(players_and_player_items), len(s1), len(s2))

0

192530

192530

### df_player_matches vs df_hero

In [21]:
# создаём множества id героев
heroes_in_players = set(df_player_matches['hero_id'].dropna())
heroes_in_heroes = set(df_heroes['hero_id'].dropna())

# находим героев, которые встречаются в df_player_matches, но отсутствуют в df_hero
missing_heroes = heroes_in_players - heroes_in_heroes
print(len(missing_heroes))

0


### df_player_items vs df_items

In [22]:
# создаём множества id предметов
items_in_players = set(df_player_items['item_id'].dropna())
items_in_items = set(df_items['item_id'].dropna())

# находим предметы, которые встречаются в df_player_items, но отсутствуют в df_items
missing_items = items_in_players - items_in_items
print(len(missing_items))

0


## Сохраняем готовые датафреймы в csv

In [23]:
# сохраним готовые датафреймы в csv
df_matches.to_csv("matches.csv", index=False, encoding="utf-8")
df_player_matches.to_csv("players.csv", index=False, encoding="utf-8")
df_player_stat.to_csv("player_stat.csv", index=False, encoding="utf-8")
df_player_items.to_csv("player_items.csv", index=False, encoding="utf-8")
df_heroes.to_csv("heroes.csv", index=False, encoding="utf-8")
df_items.to_csv("items.csv", index=False, encoding="utf-8")

In [29]:
# Код для проверки дубликатов
#dupe_rows = df_player_items.duplicated(subset=["match_id", "player_slot", "item_slot"], keep=False)
#print("Rows involved in duplicate keys:", dupe_rows.sum())